# predicting tyming of data

In [ ]:
# pip install scikit-optimize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/MyDrive/'Colab Notebooks'/eecs545

In [ ]:
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

np.random.seed(1234)

%matplotlib inline

## Prepare data set

In [ ]:
data = pd.read_csv("data/transient_pred/scalar_res.csv")

In [ ]:
data

In [ ]:
P1_log = pd.read_csv("data/transient_pred/T1_log.csv")

In [ ]:
P1_log

In [ ]:
Y = P1_log.iloc[0:len(data), 0:501].values

In [ ]:
data = data.iloc[:, 0:5]
sscaler = StandardScaler()
# sscaler = MinMaxScaler()
X = sscaler.fit_transform(data)

## training Model

In [ ]:
import torch
import torch.nn as nn 
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

def split(X, Y):
    Xtrain, Xtemp, Ytrain, Ytemp = train_test_split(X, Y, train_size = .7)
    Xtest, Xvalid, Ytest, Yvalid = train_test_split(Xtemp, Ytemp, train_size= .5)
    return Xtrain, Xvalid, Xtest, Ytrain, Yvalid, Ytest

In [ ]:
Xtrain, Xvalid, Xtest, Ytrain, Yvalid, Ytest = split(X, Y)

In [ ]:
Xtrain, Xvalid, Xtest, Ytrain, Yvalid, Ytest = split(X, Y)

X_train = torch.from_numpy(Xtrain.astype(np.float32))
y_train = torch.from_numpy(Ytrain.astype(np.float32))
X_valid = torch.from_numpy(Xvalid.astype(np.float32))
y_valid = torch.from_numpy(Yvalid.astype(np.float32))
X_test = torch.from_numpy(Xtest.astype(np.float32))
y_test = torch.from_numpy(Ytest.astype(np.float32))

train = TensorDataset(X_train,y_train)
valid = TensorDataset(X_valid,y_valid)
test = TensorDataset(X_test,y_test)

In [ ]:
my_dataloader = DataLoader(train) 

In [ ]:
input_size = X.shape[1]
output_size = Y.shape[1]

In [ ]:
trainloader = torch.utils.data.DataLoader(train, batch_size=300, shuffle=True, num_workers=1)
validloader = torch.utils.data.DataLoader(valid, batch_size=300, shuffle=False, num_workers=1)
testloader = torch.utils.data.DataLoader(test, batch_size=300, shuffle=False, num_workers=1)

In [ ]:
class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(input_size, 64),
      nn.BatchNorm1d(64),
      nn.Dropout(0.5),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.BatchNorm1d(32),
      nn.Dropout(0.5),
      nn.ReLU(),
      nn.Linear(32, 32),
      nn.BatchNorm1d(32),
      nn.Dropout(0.5),
      nn.ReLU(),
      nn.Linear(32, output_size)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

def train_model(dataloader, model, loss_fn, optimizer, epochs):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epochs % 10 == 0:
            if batch % 10 == 0:
                loss, current = loss.item(), batch * len(X)
                print(f"Train loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_model(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
    test_loss /= num_batches
    test_loss /= output_size
    print(f"Val Error: \n Avg loss: {test_loss:>8f} \n")

def predict_model(X, model):
    with torch.no_grad():
        return model(X)

In [ ]:
# Initialize the MLP
mlp = MLP()

# Define the loss function and optimizer
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-2)

In [ ]:
epochs = 50
for t in range(epochs):
    if (t % 10 ==0):
        print(f"Epoch {t+1}\n-------------------------------")
    train_model(trainloader, mlp, loss_function, optimizer, t)
    if (t % 10 ==0):
        test_model(validloader, mlp, loss_function)
print("Done!")

In [ ]:
y_test

In [ ]:
(loss_function(y_test, mlp(X_test)) / output_size ) ** 0.5

In [ ]:
pred = predict_model(X_test, mlp)
pred

In [ ]:
t = range(output_size)

In [ ]:
pred[1]

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(t, y_test[1], label="actual")
ax.plot(t, pred[1], label="predict")
ax.set_xlabel('Time')
ax.set_ylabel('Value')
plt.legend()
plt.tight_layout()

In [ ]:
def MAPE(true, pred):
    return [(np.abs(pred[i] - y_test[i]).mean() / y_test[i].mean()).item() for i in range(len(true))]

In [ ]:
np.mean(MAPE(y_test, pred))

In [ ]:
MAPE(y_test, pred)